Data Generation

In [8]:
import numpy as np
from geomdl import BSpline
from geomdl import utilities
from geomdl import exchange
from geomdl.visualization import VisPlotly as vis
from geomdl import CPGen
from matplotlib import cm
import random

In [9]:
#Helper Functions

#from https://stackoverflow.com/questions/8487893/generate-all-the-points-on-the-circumference-of-a-circle
def PointsInCircum(r,z,n=15):
    return [[np.cos(2*np.pi/n*x)*r,np.sin(2*np.pi/n*x)*r, float(z)] for x in range(0,n+1)]

Generate Geometry

In [10]:
#Initialize vertices of bottle

height = 30
radius = 10

#the mouth
mouthVertices= [PointsInCircum(3,-1)]
topVertices= [PointsInCircum(radius,height)]

variableVertices = []
for level in range(height):
    nextVertices = PointsInCircum(radius,level)
    # n = nextVertices[0]
    # nextVertices.append(n)
    for i in range(len(nextVertices) - 1):
        factor = random.random()
        nextVertices[i][0] *= factor
        nextVertices[i][1] *= factor
        if (i == 0):
            nextVertices[-1][0] *= factor
            nextVertices[-1][1] *= factor


    variableVertices += [nextVertices]

allVertices = mouthVertices + variableVertices + topVertices


Preview Geometry

In [11]:
# Create a BSpline surface instance
surf = BSpline.Surface()

# Set degrees
surf.degree_u = 3
surf.degree_v = 3

# Get the control points from the generated grid
surf.ctrlpts2d = allVertices

# Set knot vectors
surf.knotvector_u = utilities.generate_knot_vector(surf.degree_u, surf.ctrlpts_size_u)
surf.knotvector_v = utilities.generate_knot_vector(surf.degree_v, surf.ctrlpts_size_v)

# Set sample size
surf.sample_size = 100

# Set visualization component
surf.vis = vis.VisSurface(ctrlpts=False, legend=False)

# Plot the surface
surf.render(colormap=cm.terrain)

Save Geometry

In [12]:
exchange.export_obj(surf, "surf_obj.obj")
exchange.export_stl(surf, "surf_stl.stl")
print("Surface successfully exported!")

Surface successfully exported!
